In [56]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from ctgan import CTGAN

In [57]:
df = pd.read_csv("../ex1/regression/cars_optimized.csv")
df.drop("Unnamed: 0",axis=1, inplace=True)

In [58]:
df

,Prod. year,Leather interior,Mileage,Cylinders,Wheel,price_levy_combined,Airbags_bins,Engine volume int,Turbo,Coupe,...,Petrol,Automatic,Manual,Variator,4x4,Front,Germany,Japan,South Korea,USA
0,2010,0,186005,6.0,0,14727,2,3.5,0.0,0,...,0,1,0,0,1,0,0,1,0,0
1,2011,1,192000,6.0,0,17639,2,3.0,0.0,0,...,1,0,0,0,1,0,0,0,0,1
2,2006,1,200000,4.0,1,8467,1,1.3,0.0,0,...,1,0,0,1,0,1,0,1,0,0
3,2011,0,168966,4.0,0,4469,0,2.5,0.0,0,...,0,1,0,0,1,0,0,0,0,1
4,2014,0,91901,4.0,0,12172,1,1.3,0.0,0,...,1,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13255,2011,0,307325,4.0,0,1115,2,1.8,0.0,0,...,0,1,0,0,0,1,0,1,0,0
13256,1999,0,300000,4.0,0,8467,1,2.0,1.0,1,...,0,0,1,0,0,0,1,0,0,0
13257,2011,0,161600,4.0,0,16512,2,2.4,0.0,0,...,1,0,0,0,0,1,0,0,1,0
13258,2010,0,116365,4.0,0,26944,1,2.0,0.0,0,...,0,1,0,0,0,1,0,0,1,0


In [59]:
df.describe()

,Prod. year,Leather interior,Mileage,Cylinders,Wheel,price_levy_combined,Airbags_bins,Engine volume int,Turbo,Coupe,...,Petrol,Automatic,Manual,Variator,4x4,Front,Germany,Japan,South Korea,USA
count,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,...,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.000000,13260.00000
mean,2011.139668,0.310558,126074.795249,4.373605,0.070965,18249.399472,1.371795,2.156116,0.089819,0.030845,...,0.561312,0.671041,0.098039,0.049095,0.151508,0.745098,0.203469,0.350302,0.275113,0.14917
std,4.996321,0.462740,78817.455981,0.877141,0.256777,13357.233521,0.619409,0.694970,0.285933,0.172903,...,0.496245,0.469853,0.297379,0.216075,0.358557,0.435823,0.402594,0.477082,0.446588,0.35627
min,1996.000000,0.000000,0.000000,4.000000,0.000000,1001.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,2009.000000,0.000000,67680.000000,4.000000,0.000000,8774.500000,1.000000,1.600000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,2012.000000,0.000000,119450.000000,4.000000,0.000000,15681.000000,1.000000,2.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000
75%,2015.000000,1.000000,173282.250000,4.000000,0.000000,23632.000000,2.000000,2.500000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.00000
max,2020.000000,1.000000,398069.000000,8.000000,1.000000,74809.000000,3.000000,6.700000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [60]:
# Variance of original dataset
df.var()

Prod. year             2.496322e+01
Leather interior       2.141279e-01
Mileage                6.212191e+09
Cylinders              7.693764e-01
Wheel                  6.593421e-02
price_levy_combined    1.784157e+08
Airbags_bins           3.836678e-01
Engine volume int      4.829832e-01
Turbo                  8.175772e-02
Coupe                  2.989551e-02
Goods wagon            1.368480e-02
Hatchback              1.370819e-01
Jeep                   1.962370e-01
Microbus               1.265710e-02
Minivan                3.236541e-02
Pickup                 2.407631e-03
Sedan                  2.478885e-01
Universal              2.009356e-02
Diesel                 1.643922e-01
Hybrid                 1.395520e-01
LPG                    3.628828e-02
Petrol                 2.462594e-01
Automatic              2.207617e-01
Manual                 8.843420e-02
Variator               4.668822e-02
4x4                    1.285632e-01
Front                  1.899413e-01
Germany                1.620

In [61]:
fig = px.histogram(df, x="price_levy_combined")
fig.show()

My plan for fixing distribution of this dataset is : 

1. Generate more data for cars with price in between 15-30k

2. Use undersampling to reduce amount of cheap cars under 5k

3. Try Autoencoders for overlap managment

4. Run RANSAC for noise

5. Cutoff more outliers from right side of distribution



In [62]:
# Initialize CTGAN
ctgan = CTGAN()

def synthesize_data(subset, size, epochs=100):
    """"
    Fits subset of DF to CTGAN, returns a synthesized dataframe of needed size
    """
    ctgan.fit(subset_df, epochs=epochs)
    synthetic_df = ctgan.sample(size)
    return synthetic_df

In [63]:
def comparison_plot(original_df, synthesized_df, target="price_levy_combined"):
    """
    Plots a comparison of two datasets target variable distribution 
    """
    # Create histograms for both datasets
    histogram1 = go.Histogram(
        x=original_df[target],
        name='Original',
    )
    histogram2 = go.Histogram(
        x=synthesized_df[target],
        name='Synthesized',
        # Opacity so we can see original dataset
        opacity=0.5,
    )

    # Create the layout to ensure both histograms appear on the same axis
    layout = go.Layout(
        title='Comparison of original and synthesized datasets',
        barmode='overlay',
        xaxis=dict(title='Value'),
        yaxis=dict(title='Frequency'),
    )
    # Combine the histograms and layout into a figure
    fig = go.Figure(data=[histogram1, histogram2], layout=layout)
    # Show the figure
    fig.show()

In [64]:
# Create subset
subset_df = df[(df[target] > 8000) & (df[target] < 21000)]
# Generate data
synthetic_df = synthesize_data(subset_df, 3000)
# Combine with original data
synthetic_df = pd.concat([df, synthetic_df])
# plot the difference
comparison_plot(df, synthetic_df)